In [3]:
!pip install syft

     |████████████████████████████████| 296kB 2.9MB/s 
     |████████████████████████████████| 2.1MB 37.0MB/s 
     |████████████████████████████████| 256kB 48.3MB/s 
     |████████████████████████████████| 460kB 42.3MB/s 
     |████████████████████████████████| 389kB 34.9MB/s 
     |████████████████████████████████| 81kB 24.3MB/s 
     |████████████████████████████████| 204kB 29.7MB/s 
     |████████████████████████████████| 266kB 42.3MB/s 
     |████████████████████████████████| 51kB 17.7MB/s 
     |████████████████████████████████| 122kB 51.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076247 sha256=47aae3b6c884f81e03c0bab516c4d3c300dafe4401e317b18a2ac2d1516d575a
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=413341cc487df6ba5850c6056ff927a9367497bf8257cfd80814492695ab8bfc
  Stored in di

In [0]:
import torch 

In [0]:
def get_parallel_db(db, remove_index):
  return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
 def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries)>0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [0]:
def sensitivity(query, n_entries=1000):
  db, pdbs = create_db_and_parallels(n_entries)
  
  full_db_result=query(db)
  
  max_distance = 0

  for pdb in pdbs:
    pdb_result = query(pdb)
  
    db_distance=torch.abs(pdb_result - full_db_result)
  
    if(db_distance > max_distance):
      max_distance = db_distance
    
  return max_distance

In [0]:
def query(db, threshold=5):
  return (db.sum()> threshold).float()

In [10]:
db, pdbs = create_db_and_parallels(10)
db.sum()

tensor(5)

In [11]:
for i in range(10):
  sens_f = sensitivity(query, n_entries=10)
  print(sens_f)

0
0
0
tensor(1.)
0
tensor(1.)
tensor(1.)
0
tensor(1.)
tensor(1.)


In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index=10)

In [14]:
db[10]

tensor(0, dtype=torch.uint8)

In [15]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [16]:
# differencing attack using mean query

(sum(db).float() / len(db)) - (sum(pdb.float()/ len(pdb)))

tensor(-0.0045)

In [17]:
# differencing attack using threshold query

(sum(db).float() > 49 ) - (sum(pdb.float()> 49 ))

tensor(0, dtype=torch.uint8)

In [0]:
epsilon = 0.5

In [0]:
import numpy as np

In [0]:
db , pdbs = create_db_and_parallels(100)

In [0]:
def sum_query(db):
  return db.sum()

In [0]:
def laplacian_mechanism(db, query, sensitivity):
  beta = sensitivity / epsilon
  noise = torch.tensor(np.random.laplace(0,beta,1))
  

  return query(db)+ noise

**Implement local differential Privacy**

In [0]:
db , pdbs = create_db_and_parallels(100)

In [52]:
true_result= torch.mean(db.float())
true_result

tensor(0.5700)

In [0]:
def query(db):
  true_result= torch.mean(db.float())
  
  first_coin_flip= (torch.rand(len(db))>0.5).float()
  second_coin_flip= (torch.rand(len(db))>0.5).float()

  augmented_data=db.float()*first_coin_flip+ (1-first_coin_flip) * second_coin_flip
  
  db_result=torch.mean(augmented_data.float()) * 2 - 0.5
  
  return db_result, true_result

In [72]:
db , pdbs = create_db_and_parallels(10)

private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("wihtout Noise" + str(true_result))



With Noise:tensor(0.1000)
wihtout Noisetensor(0.4000)


In [73]:
db , pdbs = create_db_and_parallels(1000)

private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("wihtout Noise" + str(true_result))



With Noise:tensor(0.4960)
wihtout Noisetensor(0.5000)


In [74]:
db , pdbs = create_db_and_parallels(10000)

private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("wihtout Noise" + str(true_result))



With Noise:tensor(0.4780)
wihtout Noisetensor(0.4782)
